In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


df = pd.read_stata('./fwd_holding_merged_10q1-23q2.dta')
df = df[[
    'seriescode', 'currency', 'yq', 'curPur_usd', 'curSold_usd', 'netassets'
]]
df['net_forward_sale'] = (df['curSold_usd'] - df['curPur_usd']) / df['netassets']


df_plot = df.groupby(['currency', 'yq'])[['net_forward_sale']].sum().reset_index()

def area_plot(df_plot):
    # Define the list of currencies
    currency = ['BRL', 'CLP', 'CNY', 'CNH', 'COP', 'CZK', 'HUF', 'IDR', 'ILS', 'INR', 'KRW', 'MXN', 'MYR', 'PEN', 'PHP', 'PLN', 'RUB', 'THB', 'TRY', 'TWD', 'ZAR']
    df_plot = df_plot[df_plot['currency'].isin(currency)]

    
    fig = px.area(df_plot, x="yq", y="net_forward_sale", color="currency")

    # Initialize visibility for all currencies as True 
    visibility = [True] * len(fig.data)

    # Update layout for currency selection
    fig.update_layout(
        title="Net Position by Currency Over Time",
        xaxis_title="Time (yq)",
        yaxis_title="Net Forward Sale Position by Currency",
        template="plotly_white",
        updatemenus=[
            {
                'buttons': [
                    {
                        'label': "ALL",
                        'method': 'update',
                        'args': [
                            {'visible': [True] * len(fig.data)},
                            {'title': "Net Position by All Currencies Over Time"}
                        ]
                    },
                    *[
                        {
                            'label': currency,
                            'method': 'update',
                            'args': [
                                {'visible': [currency == selected_currency for selected_currency in df_plot['currency'].unique()]},
                                {'title': f"Net Position by {currency} Over Time"}
                            ]
                        }
                        for currency in sorted(df_plot['currency'].unique())
                    ]
                ],
                'direction': 'down',
                'showactive': True,
                'active': 0,
                'x': 0.25,
                'xanchor': 'left',
                'y': 1.15,
                'yanchor': 'top',
            }
        ]
    )
    fig.show()

    
    fig.write_html("area_chart.html", include_plotlyjs="cdn")

def bar_plot(df_plot):
    df_plot['select_date'] = pd.to_datetime(df_plot['yq'], format='%Y%m%d').dt.to_period('Q').astype(str)
    df_plot = df_plot.sort_values('select_date')

    # Filter to keep only 'Q4' data
    df_plot = df_plot[df_plot['select_date'].str.endswith('Q4')]
    df_plot['select_date'] = df_plot['select_date'].str.replace('Q4', '')

    # Create bar plot
    bar_fig = px.bar(
        df_plot,
        x="net_forward_sale",
        y="currency",
        color="select_date",
        orientation="h",
        title="Net Forward Sale Position by Date"
    )

    # Hide all bars initially
    for trace in bar_fig.data:
        trace.visible = "legendonly"

    # Update layout
    bar_fig.update_layout(
        xaxis_title="Net Forward Sale",
        yaxis_title="Currency",
        template="plotly_white",
        updatemenus=[
            {
                'buttons': [
                    {
                        'label': "Reset",
                        'method': 'update',
                        'args': [
                            {'visible': ["legendonly"] * len(bar_fig.data)},
                            {'title': "Net Position Sale Position by Date"}
                        ]
                    },
                    *[
                        {
                            'label': select_date,
                            'method': 'update',
                            'args': [
                                {'visible': [select_date == trace.name for trace in bar_fig.data]},
                                {'title': f"Net Forward Sale Position by Date - {select_date}"}
                            ]
                        }
                        for select_date in sorted(df_plot['select_date'].unique())
                    ]
                ],
                'direction': 'down',
                'showactive': True,
                'active': 0,
                'x': 0.28,
                'xanchor': 'left',
                'y': 1.27,
                'yanchor': 'top',
            }
        ]
    )

    bar_fig.show()
    
    bar_fig.write_html("bar_chart.html", include_plotlyjs="cdn")


    
area_plot(df_plot)
bar_plot(df_plot)
    
    
    